In [ ]:
# transform.py (Versão com Cálculo de DY Padronizado)
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

def classify_stock_profile(price: float, market_cap: float) -> str:
    """
    Classifica a ação com base no Market Cap e Preço.
    """
    if price is not None and price < 1.0:
        return "Penny Stock"
    if market_cap is None or market_cap == 0:
        return "N/A"
    if market_cap > 50_000_000_000:
        return "Blue Chip"
    if market_cap > 10_000_000_000:
        return "Mid Cap"
    if market_cap > 2_000_000_000:
        return "Small Cap"
    return "Micro Cap"

def get_market_sentiment_and_details(ticker_obj: yf.Ticker) -> dict:
    """
    Usa 'recommendations_summary' para maior robustez.
    """
    sentiment_data = {
        'Sentimento Gauge': 50.0, 'Strong Buy': 0, 'Buy': 0,
        'Hold': 0, 'Sell': 0, 'Strong Sell': 0
    }
    try:
        summary = ticker_obj.recommendations_summary
        if summary is None or summary.empty:
            return sentiment_data
        rec_counts = summary.iloc[-1]
        strong_buy = int(rec_counts.get('strongBuy', 0))
        buy = int(rec_counts.get('buy', 0))
        hold = int(rec_counts.get('hold', 0))
        sell = int(rec_counts.get('sell', 0))
        strong_sell = int(rec_counts.get('strongSell', 0))
        sentiment_data.update({
            'Strong Buy': strong_buy, 'Buy': buy, 'Hold': hold,
            'Sell': sell, 'Strong Sell': strong_sell
        })
        weighted_score = (strong_buy * 2) + (buy * 1) + (hold * 0) + (sell * -1) + (strong_sell * -2)
        total_recommendations = strong_buy + buy + hold + sell + strong_sell
        if total_recommendations == 0:
            return sentiment_data
        avg_score = weighted_score / total_recommendations
        normalized_sentiment = ((avg_score + 2) / 4) * 100
        sentiment_data['Sentimento Gauge'] = normalized_sentiment
        return sentiment_data
    except Exception as e:
        print(f" (Aviso: Falha ao buscar sentimento - {e}) ", end="")
        return sentiment_data

def fetch_stock_data(ticker: str, info_original: pd.Series) -> dict:
    """
    Busca e processa os dados de um único ticker, com lógica de fallback para Market Cap e DY 5 anos.
    """
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="5y")

    market_cap = info.get('marketCap')
    current_price = info.get("currentPrice", 0.0)
    if not market_cap or market_cap == 0:
        shares_outstanding = info.get('sharesOutstanding', 0)
        if shares_outstanding > 0 and current_price > 0:
            market_cap = shares_outstanding * current_price
    if not market_cap or market_cap == 0:
        market_cap = info_original.get('Market Cap', 0)

    # --- Cálculos de Indicadores ---
    # Dividend Yield 12 meses (já em percentual, apenas valida)
    dy_12m = info.get("trailingAnnualDividendYield", 0.0) * 100 if info.get("trailingAnnualDividendYield") is not None else 0.0
    if dy_12m > 50 or dy_12m < 0:
        dy_12m = 0.0  # Proteção contra valores absurdos

    # Dividend Yield 5 anos (evitar multiplicação por 100 e adicionar fallback)
    dy_5y = info.get("fiveYearAvgDividendYield", 0.0) if info.get("fiveYearAvgDividendYield") is not None else 0.0
    # Validação: valores fora de 0-50% são considerados inválidos
    if dy_5y > 50 or dy_5y < 0:
        print(f" (Aviso: DY 5 anos de {ticker} ({dy_5y:.2f}%) inválido, calculando manualmente...) ", end="")
        dy_5y = 0.0
        try:
            dividends = stock.dividends
            if not dividends.empty:
                # Soma os dividendos dos últimos 5 anos
                dividends_5y = dividends[dividends.index >= (datetime.now() - pd.Timedelta(days=5*365))]
                total_dividends = dividends_5y.sum()
                # Calcula preço médio dos últimos 5 anos
                avg_price = history['Close'].mean() if not history.empty else current_price
                if avg_price > 0:
                    dy_5y = (total_dividends / 5) / avg_price * 100  # Média anual de dividendos / preço médio
        except Exception as e:
            print(f" (Aviso: Falha ao calcular DY 5 anos manualmente para {ticker} - {e}) ", end="")

    growth_price = 0.0
    if not history.empty and len(history['Close']) > 1 and history['Close'].iloc[0] > 0:
        growth_price = ((history['Close'].iloc[-1] / history['Close'].iloc[0]) - 1) * 100
    payout_ratio = info.get("payoutRatio", 0.0) * 100 if info.get("payoutRatio") is not None else 0.0
    sentiment_info = get_market_sentiment_and_details(stock)

    dados = {
        "Empresa": info_original.get('Empresa', 'N/A'),
        "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'),
        "Tipo": info_original.get('Tipo', 'N/A'),
        "Market Cap": market_cap,
        "Logo": info_original.get('Logo', 'N/A'),
        "Preço Atual": current_price,
        "P/L": info.get("trailingPE"),
        "P/VP": info.get("priceToBook"),
        "DY (Taxa 12m, %)": dy_12m,
        "DY 5 Anos Média (%)": dy_5y,
        "Último Dividendo": info.get("lastDividendValue"),
        "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s', errors='coerce'),
        "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s', errors='coerce'),
        "Payout Ratio (%)": payout_ratio,
        "Crescimento Preço (%)": growth_price,
        "ROE (%)": info.get("returnOnEquity", 0.0) * 100 if info.get("returnOnEquity") is not None else 0.0,
        "Dívida Total": info.get('totalDebt'),
        "EBITDA": info.get('ebitda'),
        "Perfil da Ação": classify_stock_profile(current_price, market_cap)
    }
    dados.update(sentiment_info)
    if dados.get("EBITDA") and dados["EBITDA"] != 0 and dados.get("Dívida Total"):
        dados["Dívida/EBITDA"] = dados["Dívida Total"] / dados["EBITDA"]
    else:
        dados["Dívida/EBITDA"] = None
    return dados

def main():
    arquivo_input = r"E:\Github\Unicamp\Bussola-de-Valor\data\scanner_acoes_e_fundos_filtrado.csv"
    output_path = r"E:\Github\Unicamp\Bussola-de-Valor\data\relatorio_analise_b3.csv"
    try:
        df_input = pd.read_csv(arquivo_input)
        tickers_para_analisar = df_input['Ticker'].tolist()
    except FileNotFoundError:
        print(f"❌ ERRO CRÍTICO: Arquivo de entrada '{arquivo_input}' não encontrado.")
        return
    dados_detalhados = {}
    erros = []
    print("\nIniciando coleta de dados via yfinance...")
    for ticker in tickers_para_analisar:
        print(f"Processando: {ticker}...", end="")
        try:
            info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
            dados_detalhados[ticker] = fetch_stock_data(ticker, info_original)
            print(" ✅")
        except Exception as e:
            erros.append(ticker)
            print(f" ❌ (Erro: {e})")
    if not dados_detalhados:
        print("\n❌ NENHUM DADO FOI COLETADO COM SUCESSO. O arquivo CSV não será gerado.")
        return
    df_resultado = pd.DataFrame.from_dict(dados_detalhados, orient="index")
    df_resultado.fillna(0, inplace=True)
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_resultado.to_csv(output_path)
        print(f"\n✅ SUCESSO! Relatório salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"❌ ERRO INESPERADO AO SALVAR O ARQUIVO: {e}")

if __name__ == "__main__":
    main()
